**Bus Company Data Analysis**
By Edwin Kirimi Kinuthia
This are my answers for the interview questions on Data Analysis from SABABI INSTITUTE

You have recently started working as a Data Analyst for a bus company. The company wants to improve its operations, so you have been asked to analyze some data regarding the bus lines currently operated by the company. The data you will need to analyze is in this excel file: https://shorturl.at/bosK7

** Glossary **

Routes Table:
bus_num - the number of the bus line
intercity - does the bus line start in one city and end another (y = yes / n = no)
type - the type of bus service (express/ regular)
num_stops - the number of stops made by the bus
fare - the cost of a single bus ticket (the price does not change depending on where the passenger gets on/off the bus)
FOR EXAMPLE: Bus line 63 is a regular bus route with 34 stops and does not cross city lines. The fare of bus line 63 is $3.50.

Times Table:
time_of_week - is it a weekday (Mon - Fri) or weekend day (Sat - Sun)
bus_num - the number of the bus line
first_bus - the departure time of the first bus of the day (same in both directions)
last_bus - the departure time of the last bus of the day (same in both directions)
frequency - the number of times the bus completes its full route each day.
FOR EXAMPLE: On Saturdays, bus line 82 operates between 6:30 AM and 11:45 PM. During that period, it completes its route 45 times. Bus line 82 is an express buss with 9 stops and charges a fare of $12.50.

Passengers Table:
day - the day of the week
bus_num - the number of the bus line
average_num_passengers - the average number of passengers per route.
FOR EXAMPLE: Bus line 8 averages 45 passengers on Wednesdays and 15 passengers on Saturdays. Between the difference in passengers per route as well as the number of routes per day, bus line 8 generates an average of $18,000 in fares every Wednesday but only $1,500 on Saturdays.
1. On average, what is the sum total of revenue generated every Tuesday?


> Indented blockTo calculate the total revenue for Tuesday, I will use the bus line (bus_num) as the primary key. From the passengers csv table, the average number of passengers (average_num_passengers) each Tuesday for each bus line should be multiplied by the frequency of each bus line (frequency) during weekdays in the times csv table and the fare of each bus line in the routes csv table.



In [2]:
from google.colab import files
uploaded = files.upload()


Saving passengers.csv to passengers.csv
Saving routes.csv to routes.csv
Saving times.csv to times.csv


The files are now uploaded. Using the pandas module, we load the files in Colab environment.

In [4]:
import pandas as pd
passengers_df = pd.read_csv('passengers.csv')
routes_df = pd.read_csv('routes.csv')
times_df = pd.read_csv('times.csv')

Data cleaning involves removing the $ from the fare column in the routes CSV and converting the fare column to a float value for our calculations.

In [5]:
routes_df['fare'] = routes_df['fare'].str.replace('$', '').astype(float)

<ipython-input-5-fb28438ba3b4>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  routes_df['fare'] = routes_df['fare'].str.replace('$', '').astype(float)


We now Filter for Tuesday in passengers dataframe and for weekdays in the times dataframe.

In [6]:
tuesday_passengers = passengers_df[passengers_df['day'] == 'Tuesday']
weekday_times = times_df[times_df['time_of_week'] == 'weekday']

With the filtering done, we now merge both tables to create a merged dataframe.

In [7]:
merged_df = pd.merge(tuesday_passengers, routes_df, on='bus_num')
merged_df = pd.merge(merged_df, weekday_times, on='bus_num')

Calculating revenue for each bus line:

In [8]:
merged_df['revenue'] = merged_df['average_num_passengers'] * merged_df['fare'] * merged_df['frequency']

Sum total revenue for Tuesday:

In [9]:
total_tuesday_revenue = merged_df['revenue'].sum()

print(f"Total Revenue generated every Tuesday: ${total_tuesday_revenue:.2f}")

Total Revenue generated every Tuesday: $120403.00


2. Which bus line brings in the lowest average weekly revenue from ticket sales?
> Indented blockTo calculate this, using bus line as the primary key, fare in the routes csv table is multiplied by the frequency of both the weekend (Multiply this frequency by 5 since there are weekday days) and the weekday(multiply this by 2 since there are 2 days during weekends) in the times csv table and the sum of the average number of passengers from Monday to Sunday of each bus line; after comparison, the minimum value is the lowest weekly revenue.
We begin by importing the data afresh using the pandas module:



In [11]:
import pandas as pd

passengers_df = pd.read_csv('passengers.csv')
routes_df = pd.read_csv('routes.csv')
times_df = pd.read_csv('times.csv')

Cleaning the fare column in the routes dataframe to remove the dollar sign and convert it to float

In [12]:
routes_df['fare'] = routes_df['fare'].str.replace('$', '').astype(float)

<ipython-input-12-fb28438ba3b4>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  routes_df['fare'] = routes_df['fare'].str.replace('$', '').astype(float)


Aggregating passengers data by bus_num for the total average passengers per week:

In [13]:
weekly_passengers = passengers_df.groupby('bus_num')['average_num_passengers'].sum().reset_index()


Separating times data into weekdays and weekends, then adjusting frequencies:

In [14]:
times_df['adjusted_frequency'] = times_df.apply(lambda x: x['frequency'] * 5 if x['time_of_week'] == 'weekday' else x['frequency'] * 2, axis=1)
weekly_frequencies = times_df.groupby('bus_num')['adjusted_frequency'].sum().reset_index()


Merging the tables:

In [15]:
merged_df = pd.merge(weekly_passengers, routes_df, on='bus_num')
merged_df = pd.merge(merged_df, weekly_frequencies, on='bus_num')

Calculating weekly revenue for each bus line:

In [16]:
merged_df['weekly_revenue'] = merged_df['average_num_passengers'] * merged_df['fare'] * merged_df['adjusted_frequency']


Displaying the weekly revenue for all buses. Python is zero indexed so the lowest will begin at zero.

In [18]:
print(merged_df[['bus_num', 'weekly_revenue']].sort_values(by='weekly_revenue', ascending=False))

    bus_num  weekly_revenue
25       94        713550.0
16       51        543720.0
24       90        533000.0
15       46        496125.0
21       82        402187.5
4         8        396000.0
26       99        362250.0
14       42        345950.0
12       31        343562.5
3         5        322000.0
9        19        271250.0
20       77        212000.0
23       89        188125.0
7        17        175200.0
18       63        143220.0
22       85        114975.0
17       58        107065.0
13       35         92400.0
11       28         92120.0
1         2         90965.0
6        14         88567.5
2         3         87000.0
8        18         73150.0
10       22         62842.5
5        10         54827.5
19       68         48440.0
0         1         40670.0


Finding and displaying the bus line with the lowest average weekly revenue:

In [17]:
lowest_revenue_bus = merged_df.loc[merged_df['weekly_revenue'].idxmin()]

print(f"Bus line {lowest_revenue_bus['bus_num']} brings in the lowest average weekly revenue from ticket sales: ${lowest_revenue_bus['weekly_revenue']:.2f}")

Bus line 1 brings in the lowest average weekly revenue from ticket sales: $40670.00


3. On average, it costs the company $2 per stop on each route. For example, a bus line with 10 stops and has a frequency of 5 routes per day is $100 in daily operational costs. The bus company is considering shutting some lines down over the weekends. Any bus line where the weekend operational costs are more than 50% of the weekend revenue is deemed 'problematic' by the bus company.

How many problematic bus lines are there?

> Indented blockTo get the Total weekend revenue for each bus (bus line will be the primary key), the average number of passengers each bus line  on Saturday and Sunday summed (from the passengers csv table). Then the result is multiplied by fare in the routes csv table. The result is multiplied by double the frequency (double frequency since there are 2 days during weekend) from times csv table. To the operational cost is then calculated by multiplying 2 by the number of stops (num_stops) and double the frequency (for the two days in the weekend) in the routes csv table for comparison.



Loading the datasets afresh:



In [19]:
import pandas as pd

passengers_df = pd.read_csv('passengers.csv')
routes_df = pd.read_csv('routes.csv')
times_df = pd.read_csv('times.csv')

Cleaning the fare column in the routes dataframe:

In [20]:
routes_df['fare'] = routes_df['fare'].str.replace('$', '').astype(float)

<ipython-input-20-fb28438ba3b4>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  routes_df['fare'] = routes_df['fare'].str.replace('$', '').astype(float)


Aggregating weekend passenger data and sum the average passengers for Saturday and Sunday

In [24]:
weekend_passengers = passengers_df[passengers_df['day'].isin(['Saturday', 'Sunday'])]
weekend_passengers_sum = weekend_passengers.groupby('bus_num')['average_num_passengers'].sum().reset_index()

# Correctly setting weekend frequencies to avoid SettingWithCopyWarning

In [25]:
weekend_times = times_df[times_df['time_of_week'] == 'weekend'].copy()
weekend_times.loc[:, 'weekend_frequency'] = weekend_times['frequency'] * 2
weekend_frequencies = weekend_times[['bus_num', 'weekend_frequency']]

Calculating weekend revenue (average passengers * fare * doubled frequency)

In [27]:
weekend_revenue = pd.merge(weekend_passengers_sum, routes_df, on='bus_num')
weekend_revenue = pd.merge(weekend_revenue, weekend_frequencies, on='bus_num')
weekend_revenue['total_weekend_revenue'] = weekend_revenue['average_num_passengers'] * weekend_revenue['fare'] * weekend_revenue['weekend_frequency']

Calculating the operational costs (2 * num_stops * doubled frequency for the weekend)

In [28]:
weekend_revenue['operational_cost'] = 2 * weekend_revenue['num_stops'] * weekend_revenue['weekend_frequency']

Identifying problematic bus lines (where operational costs > 50% of weekend revenue)

In [29]:
weekend_revenue['is_problematic'] = weekend_revenue['operational_cost'] > (weekend_revenue['total_weekend_revenue'] * 0.5)

 Preparing and displaying the final list with all bus lines, their weekend revenue, and operational costs:

In [32]:
final_list = weekend_revenue[['bus_num', 'total_weekend_revenue', 'operational_cost']].sort_values(by='bus_num')

final_list

,bus_num,total_weekend_revenue,operational_cost
0,1,2660.0,680
1,2,9310.0,2584
2,3,12450.0,600
3,5,22500.0,1200
4,8,6600.0,1800
5,10,5775.0,1380
6,14,6090.0,2040
7,17,15000.0,960
8,18,10290.0,3360
9,19,12300.0,1800


Counting problematic bus lines:



In [30]:
problematic_lines_count = weekend_revenue['is_problematic'].sum()

Displaying problematic bus lines and the count:

In [31]:
problematic_lines = weekend_revenue[weekend_revenue['is_problematic']]

print(f"Number of problematic bus lines: {problematic_lines_count}")
problematic_lines[['bus_num', 'total_weekend_revenue', 'operational_cost']]

Number of problematic bus lines: 1


,bus_num,total_weekend_revenue,operational_cost
19,68,4095.0,2100
